In [121]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [122]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
collection = db.companies

In [123]:
# FILTER {$and: [ {deadpooled_year:null}, {founded_year: {$gt:2003}}, {offices: { $ne: [] } }, {$or: [{category_code:'games_video'},{category_code:'software'}] }]}
# PROJECT {name:1 , category_code:1, number_of_employees:1, founded_year:1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
# SORT {number_of_employees:-1}

In [124]:
def query_to_db():
    cursor = collection.find({'$and': 
                        [ {'deadpooled_year':None}, 
                          {'founded_year': {'$gt':2003}},
                          {'offices': { '$ne': [] } },
                          {'$or': [{'category_code':'games_video'},
                                   {'category_code':'software'}] 
                        }]}
                        , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                           "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                        ).sort('number_of_employees', -1)
    return cursor

In [125]:
def clean_nans(data):
    data= data.dropna(subset=['latitude', 'longitude', 'number_of_employees'])
    return data

In [126]:
def calculate_type(reg):
    if reg['founded_year'] > 2007 and reg['number_of_employees'] < 50:
        return 'Startup'
    else: return 'BigCompany'
    
def get_class(data):
    data['type']= data.apply(calculate_type, axis=1)
    return data

In [127]:
def order_data(data):
    data = data[['name', 'category_code', 'founded_year', 
                  'country_code', 'city', 'latitude', 'longitude', 
                  'number_of_employees']]
    data= data.reset_index(drop=True)
    return data

In [130]:
#Main

datos_crudo= query_to_db()
df = json_normalize(data= datos_crudo, record_path='offices', meta=['name', 'category_code', 'number_of_employees', 'founded_year'])
df_clean= clean_nans(df)
df_ordered= order_data(df_clean)
df_final= get_class(df_ordered)
df_final.to_json('data_normalize_to_plot.json', orient='records')

In [129]:
df_final


,name,category_code,founded_year,country_code,city,latitude,longitude,number_of_employees,type
0,UCWeb,software,2004,CHN,Guangzhou,23.133754,113.385331,1700,BigCompany
1,UCWeb,software,2004,USA,San Francisco,37.781476,-122.401322,1700,BigCompany
2,Veeam Software,software,2006,USA,Columbus,40.088052,-83.088566,1500,BigCompany
3,HubSpot,software,2006,USA,Cambridge,42.375392,-71.118487,650,BigCompany
4,Kabam,games_video,2006,USA,San Francisco,37.394456,-122.078531,650,BigCompany
5,Bazaarvoice,software,2005,USA,Austin,30.407545,-97.717667,600,BigCompany
6,Veeva,software,2007,USA,Pleasanton,37.692490,-121.900722,510,BigCompany
7,Zendesk,software,2007,USA,San Francisco,55.676294,12.568116,400,BigCompany
8,Dropbox,software,2007,USA,San Francisco,37.790943,-122.408499,350,BigCompany
9,SPIL GAMES,games_video,2004,NLD,Hilversum,52.220726,5.198824,300,BigCompany
